In [162]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from surprise import SVD,NMF,KNNBaseline,KNNWithMeans,SVDpp
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
np.random.seed(123)

In [150]:
# Read the ratings dataset
ratings = pd.read_csv('ml-1m/ratings.dat', delimiter='::', engine='python')
ratings = ratings.drop(columns=ratings.columns[-1])

In [151]:
# Split the dataset into test and train data such that test_data size will be 2500
train_data, test_data = train_test_split(ratings, test_size=0.0024994801)
print(f"Number of samples in test data: ",len(test_data))
print("")
# Save the test data to a seperate data structure
test_data_aside = test_data

#Save only the real ratings in a data structure for later use - because I will calculate the RMSE myself
real_ratings_test = test_data_aside.iloc[:, -1:].astype('float64')
#Save without ratings
data_no_ratings = test_data_aside.iloc[:, :-1].astype('float64')

# Create a new matrix without the test data - this will be the training data
train_matrix = ratings.drop(test_data.index)

Number of samples in test data:  2500



In [152]:
# create a dataset from the train_matrix - this is neccesary to use surprise library
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(train_matrix, reader=reader)

# SVD

In [153]:
# load SVD algorithm
algo = SVD()
# as above this is neccesary for surprise library
trainset = data.build_full_trainset()
# train the algorithm on the dataset
algo.fit(trainset)

In [154]:
# Approach 1 with 0 in the prediction set
testset = test_data.iloc[:, :2]
testset = [(row[0], row[1], 0) for row in testset.values]
# Fetch predictions
predictions = algo.test(testset)

In [155]:
#Approach 2 with real values for analysis
test_data_aside = [(row[0], row[1], row[2]) for row in test_data_aside.values]
# Fetch predictions
predictions_ = algo.test(test_data_aside)

In [156]:
# calculate the RMSE - Library approach
print("Library Approach - RMSE calculation by existing imported function: ")
accuracy.rmse(predictions_)
print("")

Library Approach - RMSE calculation by existing imported function: 
RMSE: 0.8716



In [157]:
# RMSE calculated by me (Omri and not modules)
print("RMSE calculated by me:")
real_ratings_test = real_ratings_test.values.tolist()

# data structure to save results
save_res = []
real_rate = [] #this will be the same as real_ratings_test but without [value] per value - just value

# calculating MSE below - this is not vectorized because of issues with predictions data structure
# and anyway this takes no time so might as well do it this way and avoid a headache
squared_errors_sum = 0

for i in range(len(predictions)):
    # Get the predicted rating and true rating
    pred = predictions[i].est
    true = real_ratings_test[i]
    # Save ratings
    save_res.append(pred)
    real_rate.append(true[0])
    # Calculate the squared error
    squared_error = (pred - true[0]) ** 2
    # Add to the sum
    squared_errors_sum += squared_error

# Calculate MSE
MSE = squared_errors_sum / len(predictions)

# Calculate RMSE
rmse = MSE ** 0.5

print("RMSE: ", rmse)
print("")

print("The above approach has the same output as library approach, but I stick to the assignment guidelines of getting")
print("Predictions on a matrix that is filled with 0's, and then evaluating myself against the actual ratings")
print("The catch is that i conjecture that this is what the library approach does anyway")
print("")
print("")

RMSE calculated by me:
RMSE:  0.8715878616300883

The above approach has the same output as library approach, but I stick to the assignment guidelines of getting
Predictions on a matrix that is filled with 0's, and then evaluating myself against the actual ratings
The catch is that i conjecture that this is what the library approach does anyway




In [158]:
print("Printing matrix with predictions below (of test data)")
print("")

# Add prediction to UserId,MovieID
data_no_ratings = data_no_ratings.assign(Pred = save_res)

print(data_no_ratings)

Printing matrix with predictions below (of test data)

             1    1193      Pred
336959  1983.0  1884.0  3.423174
916721  5539.0  2542.0  3.732724
411097  2466.0  1748.0  3.269454
887800  5364.0  3556.0  3.612072
380305  2223.0  1500.0  3.358821
...        ...     ...       ...
8698      59.0  1251.0  3.371622
819603  4927.0  2628.0  3.754358
470335  2896.0  1672.0  3.976425
849840  5103.0  1206.0  4.455958
84069    550.0   293.0  3.943602

[2500 rows x 3 columns]


In [159]:
print("Printing matrix with predictions and true ratings below (of test data)")
print("")

# Add real rating to UserId,MovieID,Pred
data_no_ratings = data_no_ratings.assign(Pred = save_res)
data_w_ratings = data_no_ratings.assign(True_val = real_rate)

print(data_w_ratings)

Printing matrix with predictions and true ratings below (of test data)

             1    1193      Pred  True_val
336959  1983.0  1884.0  3.423174       5.0
916721  5539.0  2542.0  3.732724       3.0
411097  2466.0  1748.0  3.269454       4.0
887800  5364.0  3556.0  3.612072       3.0
380305  2223.0  1500.0  3.358821       3.0
...        ...     ...       ...       ...
8698      59.0  1251.0  3.371622       3.0
819603  4927.0  2628.0  3.754358       4.0
470335  2896.0  1672.0  3.976425       3.0
849840  5103.0  1206.0  4.455958       5.0
84069    550.0   293.0  3.943602       5.0

[2500 rows x 4 columns]


## Now that we did it once step by step in a modular fashion, lets create a function that recieves the desired algorithm and prints results

In [160]:
def print_res(algorithm, test_data):
    # Note that I am not commenting much here since this is almost exactly as the above code which I commented heavily
    test_data_aside = test_data
    real_ratings_test = test_data_aside.iloc[:, -1:].astype('float64')
    data_no_ratings = test_data_aside.iloc[:, :-1].astype('float64')
    train_matrix = ratings.drop(test_data.index)
    
    algo = algorithm
    trainset = data.build_full_trainset()
    algo.fit(trainset)

    #Approach 1 with 0 in the prediction set
    testset = test_data.iloc[:, :2]
    testset = [(row[0], row[1], 0) for row in testset.values]
    predictions = algo.test(testset)

    #Approach 2 with real values for analysis
    test_data_aside = [(row[0], row[1], row[2]) for row in test_data_aside.values]
    predictions_ = algo.test(test_data_aside)

    # calculate the RMSE - Library approach
    print("Library Approach - RMSE calculation by existing imported function: ")
    accuracy.rmse(predictions_)
    print("")

    #RMSE calculated by me (Omri and not modules)
    print("RMSE calculated by me:")
    real_ratings_test = real_ratings_test.values.tolist()

    save_res = []
    real_rate = [] #this will be the same as real_ratings_test but without [value] per value - just value

    squared_errors_sum = 0


    for i in range(len(predictions)):
        pred = predictions[i].est
        true = real_ratings_test[i]
        save_res.append(pred)
        real_rate.append(true[0])
        squared_error = (pred - true[0]) ** 2
        squared_errors_sum += squared_error
        
    MSE = squared_errors_sum / len(predictions)
    rmse = MSE ** 0.5
    print("RMSE: ", rmse)
    print("")
    print("The above approach has the same output as library approach, but I stick to the assignment guidelines of getting")
    print("Predictions on a matrix that is filled with 0's, and then evaluating myself against the actual ratings")
    print("The catch is that i conjecture that this is what the library approach does anyway")
    print("")
    print("")
    print("Printing matrix with predictions below (of test data)")
    print("")

    data_no_ratings = data_no_ratings.assign(Pred = save_res)
    print(data_no_ratings)
    print("Printing matrix with predictions and true ratings below (of test data)")
    print("")
    data_no_ratings = data_no_ratings.assign(Pred = save_res)
    data_w_ratings = data_no_ratings.assign(True_val = real_rate)
    print(data_w_ratings)

# KNNBaseline

In [143]:
print_res(KNNBaseline(),test_data)

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9050
Library Approach - RMSE calculation by existing imported function:  0.9049703267470137

RMSE calculated by me:
RMSE:  0.9049703267470125

The above approach has the same output as library approach, but I stick to the assignment guidelines of getting
Predictions on a matrix that is filled with 0's, and then evaluating myself against the actual ratings
The catch is that i conjecture that this is what the library approach does anyway


Printing matrix with predictions below (of test data)

             1    1193      Pred
336959  1983.0  1884.0  3.742275
916721  5539.0  2542.0  3.705716
411097  2466.0  1748.0  3.285874
887800  5364.0  3556.0  3.786194
380305  2223.0  1500.0  3.437437
...        ...     ...       ...
8698      59.0  1251.0  3.700167
819603  4927.0  2628.0  3.554005
470335  2896.0  1672.0  4.075840
849840  5103.0  1206.0  5.000000
84069    550.0   293.0  4.12

# KNNWithMeans

In [144]:
print_res(KNNWithMeans(),test_data)

Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9453
Library Approach - RMSE calculation by existing imported function:  0.9452914462200038

RMSE calculated by me:
RMSE:  0.9452914462200029

The above approach has the same output as library approach, but I stick to the assignment guidelines of getting
Predictions on a matrix that is filled with 0's, and then evaluating myself against the actual ratings
The catch is that i conjecture that this is what the library approach does anyway


Printing matrix with predictions below (of test data)

             1    1193      Pred
336959  1983.0  1884.0  3.549701
916721  5539.0  2542.0  3.825037
411097  2466.0  1748.0  3.060036
887800  5364.0  3556.0  3.670475
380305  2223.0  1500.0  3.337302
...        ...     ...       ...
8698      59.0  1251.0  3.832254
819603  4927.0  2628.0  3.379729
470335  2896.0  1672.0  3.745983
849840  5103.0  1206.0  4.600581
84069    550.0   293.0  3.834970

[2500 rows x 3 columns]
P

# NMF

In [148]:
print_res(NMF(),test_data)

Library Approach - RMSE calculation by existing imported function: 
RMSE: 0.9286

RMSE calculated by me:
RMSE:  0.9285800842890745

The above approach has the same output as library approach, but I stick to the assignment guidelines of getting
Predictions on a matrix that is filled with 0's, and then evaluating myself against the actual ratings
The catch is that i conjecture that this is what the library approach does anyway


Printing matrix with predictions below (of test data)

             1    1193      Pred
336959  1983.0  1884.0  4.068487
916721  5539.0  2542.0  3.888726
411097  2466.0  1748.0  3.187604
887800  5364.0  3556.0  3.874181
380305  2223.0  1500.0  3.173197
...        ...     ...       ...
8698      59.0  1251.0  3.527827
819603  4927.0  2628.0  3.772270
470335  2896.0  1672.0  3.779615
849840  5103.0  1206.0  5.000000
84069    550.0   293.0  4.073146

[2500 rows x 3 columns]
Printing matrix with predictions and true ratings below (of test data)

             1    119

# SVD++

In [146]:
print_res(SVDpp(),test_data)

RMSE: 0.8668
Library Approach - RMSE calculation by existing imported function:  0.8668456874528642

RMSE calculated by me:
RMSE:  0.8668456874528642

The above approach has the same output as library approach, but I stick to the assignment guidelines of getting
Predictions on a matrix that is filled with 0's, and then evaluating myself against the actual ratings
The catch is that i conjecture that this is what the library approach does anyway


Printing matrix with predictions below (of test data)

             1    1193      Pred
336959  1983.0  1884.0  3.912996
916721  5539.0  2542.0  3.158015
411097  2466.0  1748.0  3.519759
887800  5364.0  3556.0  3.919585
380305  2223.0  1500.0  2.938335
...        ...     ...       ...
8698      59.0  1251.0  3.223597
819603  4927.0  2628.0  3.688566
470335  2896.0  1672.0  3.832621
849840  5103.0  1206.0  4.820497
84069    550.0   293.0  4.287275

[2500 rows x 3 columns]
Printing matrix with predictions and true ratings below (of test data)

  

# Ensemble section
### In this section we explore combining two algorithms together at each time

Note that in the beginning of each code in each sub section there will be w1,w2, these can be changed to give more weight to first/second algorithm in the ensemble

In [200]:
def print_res_ensemble(algo1,algo2, test_data,w1,w2):
    # Note that I am not commenting much here since this is almost exactly as the above code which i commented heavily
    test_data_aside = test_data
    real_ratings_test = test_data_aside.iloc[:, -1:].astype('float64')
    data_no_ratings = test_data_aside.iloc[:, :-1].astype('float64')
    train_matrix = ratings.drop(test_data.index)
    
    
    trainset = data.build_full_trainset()
    algo1.fit(trainset)
    algo2.fit(trainset)

    testset = test_data.iloc[:, :2]
    testset = [(row[0], row[1], 0) for row in testset.values]
    predictions1 = algo1.test(testset)
    predictions2 = algo2.test(testset)
    
    ensemble_pred = [(predictions1[i].uid, predictions1[i].iid, (predictions1[i].est*w1 + predictions2[i].est*w2)) for i in range(len(test_data))]
    

    #RMSE calculated by me (Omri and not modules)
    real_ratings_test = real_ratings_test.values.tolist()

    save_res = []
    real_rate = [] #this will be the same as real_ratings_test but without [value] per value - just value

    squared_errors_sum = 0


    for i in range(len(predictions)):
        pred = ensemble_pred[i][2]
        true = real_ratings_test[i]
        save_res.append(pred)
        real_rate.append(true[0])
        squared_error = (pred - true[0]) ** 2
        squared_errors_sum += squared_error
        
    MSE = squared_errors_sum / len(ensemble_pred)
    rmse = MSE ** 0.5
    print("RMSE: ", rmse)
    print("Printing matrix with predictions below (of test data)")
    print("")

    data_no_ratings = data_no_ratings.assign(Pred = save_res)
    print(data_no_ratings)
    print("Printing matrix with predictions and true ratings below (of test data)")
    print("")
    data_no_ratings = data_no_ratings.assign(Pred = save_res)
    data_w_ratings = data_no_ratings.assign(True_val = real_rate)
    print(data_w_ratings)

In [201]:
svd = SVD()
nmf = NMF()
knnbase = KNNBaseline()
knnmean = KNNWithMeans()
svdpp = SVDpp()

## SVD + NMF

In [202]:
w1 = 0.5
w2 = 0.5
print_res_ensemble(svd,nmf,test_data,w1,w2)

RMSE:  0.8786962949139095
Printing matrix with predictions below (of test data)

             1    1193      Pred
336959  1983.0  1884.0  4.144815
916721  5539.0  2542.0  3.503192
411097  2466.0  1748.0  3.138109
887800  5364.0  3556.0  3.633866
380305  2223.0  1500.0  3.210406
...        ...     ...       ...
8698      59.0  1251.0  3.463774
819603  4927.0  2628.0  3.809834
470335  2896.0  1672.0  3.849093
849840  5103.0  1206.0  4.884576
84069    550.0   293.0  4.194686

[2500 rows x 3 columns]
Printing matrix with predictions and true ratings below (of test data)

             1    1193      Pred  True_val
336959  1983.0  1884.0  4.144815       5.0
916721  5539.0  2542.0  3.503192       3.0
411097  2466.0  1748.0  3.138109       4.0
887800  5364.0  3556.0  3.633866       3.0
380305  2223.0  1500.0  3.210406       3.0
...        ...     ...       ...       ...
8698      59.0  1251.0  3.463774       3.0
819603  4927.0  2628.0  3.809834       4.0
470335  2896.0  1672.0  3.849093       

## SVD + KNNBase

In [197]:
w1 = 0.5
w2 = 0.5
print_res_ensemble(svd,knnbase,test_data,w1,w2)

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE:  0.8734560161756824
Printing matrix with predictions below (of test data)

             1    1193      Pred
336959  1983.0  1884.0  3.614364
916721  5539.0  2542.0  3.659183
411097  2466.0  1748.0  3.386835
887800  5364.0  3556.0  3.814199
380305  2223.0  1500.0  3.362490
...        ...     ...       ...
8698      59.0  1251.0  3.489179
819603  4927.0  2628.0  3.666225
470335  2896.0  1672.0  3.967010
849840  5103.0  1206.0  4.740992
84069    550.0   293.0  4.245054

[2500 rows x 3 columns]
Printing matrix with predictions and true ratings below (of test data)

             1    1193      Pred  True_val
336959  1983.0  1884.0  3.614364       5.0
916721  5539.0  2542.0  3.659183       3.0
411097  2466.0  1748.0  3.386835       4.0
887800  5364.0  3556.0  3.814199       3.0
380305  2223.0  1500.0  3.362490       3.0
...        ...     ...       ...       ...
8698      59.0  1251.

## SVD + KNNmean

In [198]:
w1 = 0.5
w2 = 0.5
print_res_ensemble(svd,knnmean,test_data,w1,w2)

Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE:  0.8826589030893733
Printing matrix with predictions below (of test data)

             1    1193      Pred
336959  1983.0  1884.0  3.651859
916721  5539.0  2542.0  3.832160
411097  2466.0  1748.0  3.130887
887800  5364.0  3556.0  3.524966
380305  2223.0  1500.0  3.368824
...        ...     ...       ...
8698      59.0  1251.0  3.504920
819603  4927.0  2628.0  3.561740
470335  2896.0  1672.0  3.911558
849840  5103.0  1206.0  4.504979
84069    550.0   293.0  3.932813

[2500 rows x 3 columns]
Printing matrix with predictions and true ratings below (of test data)

             1    1193      Pred  True_val
336959  1983.0  1884.0  3.651859       5.0
916721  5539.0  2542.0  3.832160       3.0
411097  2466.0  1748.0  3.130887       4.0
887800  5364.0  3556.0  3.524966       3.0
380305  2223.0  1500.0  3.368824       3.0
...        ...     ...       ...       ...
8698      59.0  1251.0  3.504920       3.0
819603  4

## NMF + SVDpp

In [203]:
w1 = 0.5
w2 = 0.5
print_res_ensemble(svdpp,nmf,test_data,w1,w2)

RMSE:  0.8799423653212551
Printing matrix with predictions below (of test data)

             1    1193      Pred
336959  1983.0  1884.0  3.912680
916721  5539.0  2542.0  3.536867
411097  2466.0  1748.0  3.360398
887800  5364.0  3556.0  3.831776
380305  2223.0  1500.0  3.234815
...        ...     ...       ...
8698      59.0  1251.0  3.468135
819603  4927.0  2628.0  3.821867
470335  2896.0  1672.0  3.800453
849840  5103.0  1206.0  4.811450
84069    550.0   293.0  4.141300

[2500 rows x 3 columns]
Printing matrix with predictions and true ratings below (of test data)

             1    1193      Pred  True_val
336959  1983.0  1884.0  3.912680       5.0
916721  5539.0  2542.0  3.536867       3.0
411097  2466.0  1748.0  3.360398       4.0
887800  5364.0  3556.0  3.831776       3.0
380305  2223.0  1500.0  3.234815       3.0
...        ...     ...       ...       ...
8698      59.0  1251.0  3.468135       3.0
819603  4927.0  2628.0  3.821867       4.0
470335  2896.0  1672.0  3.800453       

## SVDpp + KNNBase

In [204]:
w1 = 0.5
w2 = 0.5
print_res_ensemble(svdpp,knnbase,test_data,w1,w2)

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE:  0.8649140663551184
Printing matrix with predictions below (of test data)

             1    1193      Pred
336959  1983.0  1884.0  3.760400
916721  5539.0  2542.0  3.488509
411097  2466.0  1748.0  3.386564
887800  5364.0  3556.0  3.772984
380305  2223.0  1500.0  3.282718
...        ...     ...       ...
8698      59.0  1251.0  3.425559
819603  4927.0  2628.0  3.677333
470335  2896.0  1672.0  3.924004
849840  5103.0  1206.0  4.921688
84069    550.0   293.0  4.196777

[2500 rows x 3 columns]
Printing matrix with predictions and true ratings below (of test data)

             1    1193      Pred  True_val
336959  1983.0  1884.0  3.760400       5.0
916721  5539.0  2542.0  3.488509       3.0
411097  2466.0  1748.0  3.386564       4.0
887800  5364.0  3556.0  3.772984       3.0
380305  2223.0  1500.0  3.282718       3.0
...        ...     ...       ...       ...
8698      59.0  1251.

# Ensemble of all algorithms together (can play with weights)
### below the function definition below, there are 5 weights, each weight corresponds to an algorithm in the order they are written, you can play around with the weights and assign each algorithm more or less weight - of course assigning an algorithm 0 weight will make it negligent - and assigning only two algorithms non zero weights will yield the above 2 pair ensembles (which is a special case of this ensemble), enjoy :)

In [205]:
def print_res_ensemble_all(algo1,algo2,algo3,algo4,algo5, test_data,w1,w2,w3,w4,w5):
    # Note that I am not commenting much here since this is almost exactly as the above code which i commented heavily
    test_data_aside = test_data
    real_ratings_test = test_data_aside.iloc[:, -1:].astype('float64')
    data_no_ratings = test_data_aside.iloc[:, :-1].astype('float64')
    train_matrix = ratings.drop(test_data.index)
    
    
    trainset = data.build_full_trainset()
    algo1.fit(trainset)
    algo2.fit(trainset)
    algo3.fit(trainset)
    algo4.fit(trainset)
    algo5.fit(trainset)

    #Approach 1 with 0 in the prediction set
    testset = test_data.iloc[:, :2]
    testset = [(row[0], row[1], 0) for row in testset.values]
    predictions1 = algo1.test(testset)
    predictions2 = algo2.test(testset)
    predictions3 = algo3.test(testset)
    predictions4 = algo4.test(testset)
    predictions5 = algo5.test(testset)

    
    ensemble_pred = [(predictions1[i].uid, predictions1[i].iid, (predictions1[i].est*w1 + predictions2[i].est*w2 + predictions3[i].est*w3 + predictions4[i].est*w4 + predictions5[i].est*w5)) for i in range(len(test_data))]
    

    #RMSE calculated by me (Omri and not modules)
    real_ratings_test = real_ratings_test.values.tolist()

    save_res = []
    real_rate = [] #this will be the same as real_ratings_test but without [value] per value - just value

    squared_errors_sum = 0


    for i in range(len(predictions)):
        pred = ensemble_pred[i][2]
        true = real_ratings_test[i]
        save_res.append(pred)
        real_rate.append(true[0])
        squared_error = (pred - true[0]) ** 2
        squared_errors_sum += squared_error
        
    MSE = squared_errors_sum / len(ensemble_pred)
    rmse = MSE ** 0.5
    print("RMSE: ", rmse)
    print("Printing matrix with predictions below (of test data)")
    print("")

    data_no_ratings = data_no_ratings.assign(Pred = save_res)
    print(data_no_ratings)
    print("Printing matrix with predictions and true ratings below (of test data)")
    print("")
    data_no_ratings = data_no_ratings.assign(Pred = save_res)
    data_w_ratings = data_no_ratings.assign(True_val = real_rate)
    print(data_w_ratings)

In [206]:
svd = SVD()
nmf = NMF()
knnbase = KNNBaseline()
knnmean = KNNWithMeans()
svdpp = SVDpp()
w1 = 0.2
w2 = 0.2
w3 = 0.2
w4 = 0.2
w5 = 0.2

In [209]:
print_res_ensemble_all(svd,nmf,knnbase,knnmean,svdpp,test_data,w1,w2,w3,w4,w5)

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE:  0.881456717902454
Printing matrix with predictions below (of test data)

             1    1193      Pred
336959  1983.0  1884.0  3.771443
916721  5539.0  2542.0  3.613028
411097  2466.0  1748.0  3.159610
887800  5364.0  3556.0  3.858112
380305  2223.0  1500.0  3.293648
...        ...     ...       ...
8698      59.0  1251.0  3.501371
819603  4927.0  2628.0  3.611931
470335  2896.0  1672.0  3.790564
849840  5103.0  1206.0  4.854605
84069    550.0   293.0  4.174607

[2500 rows x 3 columns]
Printing matrix with predictions and true ratings below (of test data)

             1    1193      Pred  True_val
336959  1983.0  1884.0  3.771443       5.0
916721  5539.0  2542.0  3.613028       3.0
411097  2466.0  1748.0  3.159610       4.0
887800  5364.0  3556.0  3.858112       3.0
380305  2223.0  1500.0  3.293648   